# Charging Model

## initialize set
- import modules
- set problems
- set idnex
- define variables
- read data from files
- define constants


In [28]:
# import Modules
import xpress as xp
import numpy as np
import pandas as pd

# use professional edition xpress package!!!

In [29]:
# Create a problem called myproblem

prob = xp.problem(name='charging')

In [30]:
# Set index
number_of_types = 3 # i: 1..3
types = range(number_of_types)

number_of_years = 4 # Time t: 1..4 0..4
years = range(number_of_years)
number_of_years2 = number_of_years + 1
years2 = range(number_of_years2)

number_of_grids = 434 # grid g: 1..434
grids = range(number_of_grids)


In [31]:
# Define our decision variable as a numpy array
x1 = np.array([xp.var( name='x1_{0}_{1}_{2}'.format(i+1,g+1,t+1),vartype = xp.integer)
                    for i in types for g in grids for t in years], dtype=xp.npvar).reshape(number_of_types,number_of_grids,number_of_years)
x2 = np.array([xp.var( name='x2_{0}_{1}_{2}'.format(i+1,g+1,t+1),vartype = xp.integer)
                    for i in types for g in grids for t in years], dtype=xp.npvar).reshape(number_of_types,number_of_grids,number_of_years)
x = np.array([xp.var( name='x_{0}_{1}_{2}'.format(i+1,g+1,t),vartype = xp.integer)
                    for i in types for g in grids for t in range(number_of_years2)], dtype=xp.npvar).reshape(number_of_types,number_of_grids,number_of_years2)
# x = x1 + x2
z = np.array([xp.var( name='z_{0}_{1}'.format(g+1,t+1),vartype = xp.binary)
                    for g in grids for t in years], dtype=xp.npvar).reshape(number_of_grids,number_of_years)

y = np.array([xp.var( name='y_{0}_{1}'.format(g+1,t+1),vartype = xp.integer)
                    for g in grids for t in years], dtype=xp.npvar).reshape(number_of_grids,number_of_years)

prob.addVariable(x1,x2,x,y,z)

In [32]:
# read data from files
pot_char_poi = pd.read_excel('Project_data/Potential_charging_points.xlsx')
intere_poi = pd.read_excel('Project_data/Interest _points.xlsx')
demand = pd.read_excel('Project_data/Demand_data.xlsx')
char_poi =  pd.read_excel('Project_data/Charging_points.xlsx')


In [33]:
#  Constant
c = np.array(types) # building cost for charging point of type i
c = [1,1,1] #!!!Notice: can define by ourselves

# the reduced building cost for charging point of type i
r = np.array(types) 
r = [0.8,0.8,0.8] #!!!Notice: can define by ourselves

# the maximal number of location in the grid square g
B = np.array(grids) 
for i in B:
    i = 3 #!!!Notice: can define by ourselves
    
# limitation of charger's number in every location
A = 10 #!!!Notice: can define by ourselves

# A*B_{g}:the maximal number of charging points in the grid square g, where 

# the lower bound of proportion for type i charging point
L = np.array(types)
L = [0.3,0.3,0.3] #!!!Notice: can define by ourselves

# the Upper bound of proportion for type i charging point
U = np.array(types) 
U = [0.7,0.7,0.7] #!!!Notice: can define by ourselves

# maximal number of station points in the centre of city.
SP = 100 #!!!Notice: can define by ourselves

# the satisfaction proportion of for the total demand
TS = 0.9 #!!!Notice: can define by ourselves

# the proportion of the least demand provided by neighbor for 1 grid.
MS = 0.8 #!!!Notice: can define by ourselves

# define which grids belong to center by "distance to center"
DCenter = 2000 #!!!Notice: can define by ourselves

# demand for the grid g in year t
d = np.array((demand['Demand_0'],demand['Demand_1'],demand['Demand_2'],demand['Demand_3']))

# the minimum demand type i can provide per year
Pi1 = np.array(years) 
Pi1 = [2000,4000,30000]

# the maximum demand type i can provide per year 
Pi = np.array(years) 
Pi = [3500,5200,55000]

# Pg: the number of potential locations in the grid square
pg = (pot_char_poi['grid number'].value_counts())
Pg = np.zeros(number_of_grids) 
for i in pg.index:
    Pg[i-1] = pg[i]
    

## Add Constraints

In [34]:
# constrain 0
# relastion between variables and exist constants
char_poi['Type'].replace('Slow',0,inplace = True)
char_poi['Type'].replace('Fast',1,inplace = True)
char_poi['Type'].replace('Rapid',2,inplace = True)
exist_charger = pd.DataFrame({'Type':char_poi['Type'],'Grid':char_poi['grid number']})


for i in types:
    for g in grids:
        prob.addConstraint( x[i,g,0] == ((exist_charger.Type == i) & (exist_charger.Grid ==g)).sum())


In [35]:
# constrain 1 
# the number of chargers cannot be more than limitation in t years in g grids

prob.addConstraint(y[g,t] <= A*B[g] for g in grids for t in years)

In [36]:
# constrian 2
# the limitation of charger number in city center
distance = demand['Distance from Centre'].copy()
distance.sort_values(inplace = True)
center = distance[distance <= DCenter]
center_grid = np.array(center.index)


prob.addConstraint((xp.Sum(y[int(g),t] for g in center_grid) <= SP) for t in years)  


In [37]:
# constrain 3
# multi chargers' types
for i in types:
    for t in years2:
        if t>0:
            prob.addConstraint(L[i]*xp.Sum(x[i1,g,t1] for i1 in types for g in grids for t1 in years2 if t1<=t) <= 
                          xp.Sum(x[i,g,t1] for g in grids for t1 in years2 if t1 <=t))
            prob.addConstraint(U[i]*xp.Sum(x[i1,g,t1] for i1 in types for g in grids for t1 in years2 if t1<=t) >= 
                          xp.Sum(x[i,g,t1] for g in grids for t1 in years2 if t1<=t))


In [38]:
# constrain 4
# the proportion of total demand at least should be satisfied(TS)

TotalDemand = np.array([np.sum(d[:][0]),np.sum(d[:][1]),np.sum(d[:][2]),np.sum(d[:][3])])
prob.addConstraint(2*xp.Sum(Pi1[i]*xp.Sum(x[i,g,t1] for g in grids) 
                          for i in types for t1 in years2 if t1<=t)>=TS*TotalDemand[t-1] for t in years2 if t>=1)


In [39]:
# constraint 5
# the proportion of 1 grid's demand satisfied by itselves and neighbors
neighbors = demand['NEIGHBORS']
for g in grids:
    neighbor = np.array(neighbors[g].strip('[]').split(','),dtype = 'int')
    neighbor = np.insert(neighbor,0,[int(demand['Ref'][g])])
    prob.addConstraint(xp.Sum(Pi1[i]*xp.Sum(x[i,n-1,t1] for n in neighbor) for i in types for t1 in years2 if t1<=t)
                      >= MS*d[t-1][g] for t in years2 if t >=1)


In [40]:
# constraint 6
# x = x1 + x2
prob.addConstraint(x[i,g,t+1] == x1[i,g,t] + x2[i,g,t] for i in types for g in grids for t in years)

In [41]:
# constraint 7
# relation between x and y 
for g in grids:
    prob.addConstraint(y[g,0] == xp.Sum((x[i,g,0]+x[i,g,1]) for i in types))
    prob.addConstraint(y[g,1] == xp.Sum((x[i,g,0]+x[i,g,1] + x[i,g,2]) for i in types))
    prob.addConstraint(y[g,2] == xp.Sum((x[i,g,0]+x[i,g,1] + x[i,g,2] + x[i,g,3]) for i in types))
    prob.addConstraint(y[g,3] == xp.Sum((x[i,g,0]+x[i,g,1] + x[i,g,2] + x[i,g,3] + x[i,g,4]) for i in types))


In [42]:
# constraint 8
# calculate reduced cost
M = 1000
# M = A*B[0]
        
prob.addConstraint((1-z[g,t]) <= y[g,t] for g in grids for t in years)
prob.addConstraint(y[g,t] <= (1-z[g,t])*M for g in grids for t in years)
prob.addConstraint(x1[i,g,t] <= M*z[g,t] for i in types for g in grids for t in years)
prob.addConstraint(x2[i,g,t] <= M*(1-z[g,t]) for i in types for g in grids for t in years)


In [43]:
# constraint 9
# binary variables' definition z \in {0,1}
# written in prebious module

In [44]:
# constraint 10
# >= 0
prob.addConstraint(x1[i,g,t] >= 0 for i in types for g in grids for t in years)
prob.addConstraint(x2[i,g,t] >= 0 for i in types for g in grids for t in years)
prob.addConstraint(y[g,t] >= 0 for g in grids for t in years)

In [45]:
# constraint 11
# chargers have to be produced in potential locations
for g in grids:
    prob.addConstraint(x[i,g,t+1] <= Pg[g]*A for i in types for t in years)


## Solving Problem

In [46]:
# Objective function
prob.setObjective(xp.Sum(c[i]*x1[i,g,t] for i in types for g in grids for t in years) 
                  + xp.Sum(r[i] * x2[i,g,t] for i in types for g in grids for t in years),
                 sense = xp.minimize)

In [47]:
prob.write("problem","lp")

In [ ]:
prob.solve()

FICO Xpress v8.13.7, Hyper, solve started 19:06:49, Nov 28, 2022
Heap usage: 25MB (peak 25MB, 9820KB system)
Minimizing MILP charging using up to 8 threads, with these control settings:
OUTPUTLOG = 1
Original problem has:
     42998 rows        20398 cols       364414 elements     20398 globals
Presolved problem has:
      1695 rows         1409 cols        15652 elements      1409 globals
LP relaxation tightened
Presolve finished in 0 seconds
Heap usage: 23MB (peak 56MB, 9820KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 3.00e-01,  6.00e+04] / [ 1.95e-03,  1.99e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  9.40e+05] / [ 1.00e+00,  1.00e+03]
  Objective      [min,max] : [ 8.00e-01,  1.00e+00] / [ 8.00e-01,  1.00e+00]
Autoscaling applied standard scaling

Symmetric problem: generators: 16, support set: 404
 Number of orbits: 146, largest orbit: 6
 Row orbits: 182, row support: 504
Will try to keep branch and bound

    3487    32.800000    32.457143      2   1191     68    1.05%       3      2
    3614    32.800000    32.457143      2   1286     98    1.05%       2      2
    3766    32.800000    32.457143      2   1294     50    1.05%       1      2
    3871    32.800000    32.457143      2   1470     38    1.05%       2      2
    3993    32.800000    32.457143      2   1470     86    1.05%       3      2
    4105    32.800000    32.457143      2   1486     56    1.05%       4      2
    4209    32.800000    32.457143      2   1494     51    1.05%       3      2
    4332    32.800000    32.457143      2   1511     64    1.05%       1      2
    4436    32.800000    32.457143      2   1556     43    1.05%       3      2
Resetting tree to root.

Performing root presolve...

Reduced problem has:     479 rows     315 columns      2802 elements
Presolve dropped   :       5 rows       3 columns        64 elements
Presolve tightened :       122 elements
Symmetric problem: generators: 24, support set: 

   10429    32.800000    32.457143      2   2421     33    1.05%      10      6
   10552    32.800000    32.457143      2   2591     54    1.05%       9      6
   10668    32.800000    32.457143      2   2642     55    1.05%       3      6
   10770    32.800000    32.457143      2   2717      1    1.05%       1      6
   10872    32.800000    32.457143      2   2769      1    1.05%       1      6
   10972    32.800000    32.457143      2   2767     28    1.05%      13      6
   11127    32.800000    32.457143      2   2765     98    1.05%       1      6
   11229    32.800000    32.457143      2   2805     88    1.05%       4      7
   11354    32.800000    32.457143      2   2959     30    1.05%       9      7
   11463    32.800000    32.457143      2   2963     89    1.05%       7      7
   11587    32.800000    32.457143      2   3002     88    1.05%       1      7
   11710    32.800000    32.457143      2   3018     63    1.05%       1      7
   11810    32.800000    32.457143      

   72600    32.800000    32.457143      2  26904     93    1.05%       4     31
Elapsed time (sec): 31, estimated tree completion: 0.04215
Heap usage: 203MB (peak 203MB, 10MB system)
B&B tree size: 78MB total
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
   73634    32.800000    32.457143      2  27355     74    1.05%       4     31
   74702    32.800000    32.457143      2  27743    101    1.05%       1     32
   75712    32.800000    32.457143      2  28136    103    1.05%       1     32
   76731    32.800000    32.457143      2  28468    109    1.05%       3     32
   77746    32.800000    32.457143      2  28923     63    1.05%       8     33
   78755    32.800000    32.457143      2  29244     84    1.05%       1     33
   79808    32.800000    32.457143      2  29698     97    1.05%       6     34
   80834    32.800000    32.457143      2  30081     96    1.05%       9     34
   81847    32.800000    32.457143      2  30540     71    1.05%     

  686521    32.800000    32.457143      2 208064     93    1.05%      10    274
  696523    32.800000    32.457143      2 209874     93    1.05%      10    279
  706529    32.800000    32.457143      2 212395     99    1.05%       5    284
  716533    32.800000    32.457143      2 214755     95    1.05%       6    288
  726534    32.800000    32.457143      2 216650     83    1.05%      10    293
  736535    32.800000    32.457143      2 218075     95    1.05%       8    297
  746566    32.800000    32.457143      2 219443     96    1.05%       7    301
  756586    32.800000    32.457143      2 221411    105    1.05%       7    305
  766591    32.800000    32.457143      2 223834    103    1.05%       3    309
  776603    32.800000    32.457143      2 226495    106    1.05%       3    313
  786604    32.800000    32.457143      2 229181     63    1.05%       9    317
Elapsed time (sec): 323, estimated tree completion: 0.04215
Heap usage: 777MB (peak 777MB, 10MB system)
B&B tree size: 0

...

 2469589    32.800000    32.457143      2 621775     89    1.05%       1    983
 2479626    32.800000    32.457143      2 624757    102    1.05%       5    987
 2489657    32.800000    32.457143      2 627204     96    1.05%       3    991
 2499657    32.800000    32.457143      2 629432     98    1.05%       5    996
 2509670    32.800000    32.457143      2 631865    107    1.05%       1    999
 2519689    32.800000    32.457143      2 634014     79    1.05%       5   1003
 2529694    32.800000    32.457143      2 636460    100    1.05%      10   1007
 2539696    32.800000    32.457143      2 639210    106    1.05%       9   1011
 2549702    32.800000    32.457143      2 641571    100    1.05%       6   1016
 2559709    32.800000    32.457143      2 643685     96    1.05%      10   1020
 2569721    32.800000    32.457143      2 644553     96    1.05%      12   1024
 2579736    32.800000    32.457143      2 646393    101    1.05%       7   1028
 2589742    32.800000    32.457143      

 3371092    32.800000    32.457143      2 829148     89    1.05%       9   1344
 3381106    32.800000    32.457143      2 830697     93    1.05%       8   1348
 3391133    32.800000    32.457143      2 832471    111    1.05%       4   1352
Elapsed time (sec): 1356, estimated tree completion: 0.04215
Heap usage: 2554MB (peak 2554MB, 10MB system)
B&B tree size: 2.1GB total
               25MB active node information
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
 3401138    32.800000    32.457143      2 834159     92    1.05%      11   1356
 3411146    32.800000    32.457143      2 835753    105    1.05%       3   1359
 3421166    32.800000    32.457143      2 837956    111    1.05%       4   1363
 3431180    32.800000    32.457143      2 839895     98    1.05%       7   1367
 3441197    32.800000    32.457143      2 841571     98    1.05%       5   1371
 3451205    32.800000    32.457143      2 842826     95    1.05%       7   1375
 3461213    32.80000

 4242542    32.800000    32.457143      2 1034983    102    1.05%       4   1676
 4252550    32.800000    32.457143      2 1037746     94    1.05%       4   1679
 4262589    32.800000    32.457143      2 1039900    103    1.05%       6   1683
 4272624    32.800000    32.457143      2 1042355    100    1.05%       1   1686
 4282629    32.800000    32.457143      2 1044761     80    1.05%       1   1690
 4292669    32.800000    32.457143      2 1046785    112    1.05%       1   1693
 4302686    32.800000    32.457143      2 1049068     93    1.05%       1   1697
 4312706    32.800000    32.457143      2 1051164    102    1.05%       1   1701
 4322710    32.800000    32.457143      2 1053375    102    1.05%       6   1704
 4332731    32.800000    32.457143      2 1055813    110    1.05%      10   1708
 4342733    32.800000    32.457143      2 1058565     90    1.05%      13   1711
 4352736    32.800000    32.457143      2 1060233    110    1.05%      10   1715
 4362737    32.800000    32.

 5134095    32.800000    32.457143      2 1221961     92    1.05%      11   2026
 5144133    32.800000    32.457143      2 1223847    105    1.05%       7   2031
 5154137    32.800000    32.457143      2 1225773     90    1.05%      11   2035
 5164137    32.800000    32.457143      2 1227795     99    1.05%       3   2039
 5174161    32.800000    32.457143      2 1229411    103    1.05%       7   2043
 5184188    32.800000    32.457143      2 1231304     97    1.05%      12   2048
 5194216    32.800000    32.457143      2 1232742    101    1.05%       3   2052
Elapsed time (sec): 2057, estimated tree completion: 0.04215
Heap usage: 3787MB (peak 3787MB, 10MB system)
B&B tree size: 3.1GB total
               38MB active node information
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
 5204228    32.800000    32.457143      2 1234368     97    1.05%       6   2057
 5214257    32.800000    32.457143      2 1236202    102    1.05%       7   2062
 5224260   

 6015408    32.800000    32.457143      2 1403755     61    1.05%       8   2403
 6025422    32.800000    32.457143      2 1405519     95    1.05%       8   2408
 6035430    32.800000    32.457143      2 1407286    103    1.05%       9   2412
 6045436    32.800000    32.457143      2 1409277     96    1.05%       5   2416
 6055436    32.800000    32.457143      2 1411403     96    1.05%       8   2420
 6065436    32.800000    32.457143      2 1413389     98    1.05%       9   2424
 6075453    32.800000    32.457143      2 1415592    106    1.05%       7   2429
 6085473    32.800000    32.457143      2 1417755    101    1.05%       6   2433
 6095481    32.800000    32.457143      2 1419781     94    1.05%       5   2437
 6105495    32.800000    32.457143      2 1422365    104    1.05%       8   2441
 6115506    32.800000    32.457143      2 1424785     98    1.05%       7   2445
 6125532    32.800000    32.457143      2 1426922     83    1.05%       5   2449
 6135549    32.800000    32.

## Result

In [ ]:
print("the objective value is: ",prob.getObjVal())

In [ ]:
X = prob.getSolution(x)
Type1 = X[0]
Type2 = X[1]
Type3 = X[2]

In [25]:
print("Slow charger :")
type1 = np.nonzero(Type1)
for i in range(len(type1[0])):
    print("set charger in grid ",type1[0][i],' in year ',type1[1][i])
print('if year equal to 0, the charger has been set previously.')

Slow charger :
set charger in grid  76  in year  0
set charger in grid  91  in year  0
set charger in grid  117  in year  0
set charger in grid  129  in year  0
set charger in grid  146  in year  0
set charger in grid  147  in year  0
set charger in grid  150  in year  0
set charger in grid  159  in year  0
set charger in grid  160  in year  0
set charger in grid  163  in year  0
set charger in grid  173  in year  0
set charger in grid  189  in year  0
set charger in grid  190  in year  0
set charger in grid  199  in year  0
set charger in grid  200  in year  0
set charger in grid  202  in year  0
set charger in grid  205  in year  0
set charger in grid  214  in year  0
set charger in grid  228  in year  0
set charger in grid  259  in year  0
set charger in grid  263  in year  0
set charger in grid  274  in year  0
set charger in grid  288  in year  0
set charger in grid  300  in year  0
set charger in grid  301  in year  0
set charger in grid  304  in year  0
set charger in grid  314 

In [26]:
print("Fast charger :")
type2 = np.nonzero(Type2)
for i in range(len(type2[0])):
    print("set charger in grid ",type2[0][i],' in year ',type2[1][i])
print('if year equal to 0, the charger has been set previously.')

Fast charger :
set charger in grid  59  in year  0
set charger in grid  60  in year  1
set charger in grid  76  in year  0
set charger in grid  101  in year  1
set charger in grid  104  in year  1
set charger in grid  106  in year  1
set charger in grid  108  in year  1
set charger in grid  115  in year  1
set charger in grid  143  in year  1
set charger in grid  144  in year  0
set charger in grid  155  in year  1
set charger in grid  161  in year  1
set charger in grid  185  in year  0
set charger in grid  199  in year  0
set charger in grid  200  in year  0
set charger in grid  214  in year  0
set charger in grid  216  in year  0
set charger in grid  229  in year  1
set charger in grid  289  in year  1
set charger in grid  345  in year  0
set charger in grid  369  in year  0
if year equal to 0, the charger has been set previously.


In [27]:
print("Rapid charger :")
type3 = np.nonzero(Type3)
for i in range(len(type3[0])):
    print("set charger in grid ",type3[0][i],' in year ',type3[1][i])
print('if year equal to 0, the charger has been set previously.')

Rapid charger :
set charger in grid  60  in year  1
set charger in grid  90  in year  1
set charger in grid  103  in year  0
set charger in grid  114  in year  1
set charger in grid  118  in year  1
set charger in grid  120  in year  1
set charger in grid  134  in year  0
set charger in grid  170  in year  1
set charger in grid  175  in year  1
set charger in grid  186  in year  0
set charger in grid  190  in year  0
set charger in grid  199  in year  0
set charger in grid  212  in year  1
set charger in grid  216  in year  0
set charger in grid  232  in year  1
set charger in grid  245  in year  1
set charger in grid  248  in year  1
set charger in grid  263  in year  0
set charger in grid  271  in year  1
set charger in grid  287  in year  1
set charger in grid  290  in year  1
set charger in grid  298  in year  1
set charger in grid  326  in year  1
set charger in grid  327  in year  1
set charger in grid  355  in year  1
set charger in grid  368  in year  1
set charger in grid  369